In [1]:
import os
import cv2
import numpy as np
import shutil
from deepface import DeepFace
import threading
import time
import queue

## Tools & tests

Creating samples folder (celebrities)

In [9]:
data_dir = './Datasets/famosos'

for directory in os.listdir(data_dir):
    first_file = os.listdir(os.path.join(data_dir, directory))[0]
    shutil.copyfile(os.path.join(data_dir, directory, first_file), os.path.join("Samples", f"{directory}.jpg"))

Lookalike test

In [13]:
result = DeepFace.verify("person1.jpg", f"Samples/Angelina Jolie.jpg")

In [14]:
result

{'verified': False,
 'distance': 0.7698307337090413,
 'threshold': 0.4,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 208, 'y': 37, 'w': 120, 'h': 120},
  'img2': {'x': 74, 'y': 115, 'w': 287, 'h': 287}},
 'time': 0.54}

Age, render & race test

In [5]:
emotionImg = cv2.imread('./person1.jpg')
obj = DeepFace.analyze(emotionImg, enforce_detection=False, actions=['age', 'gender', 'race', 'emotion'])

print(obj)

Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.77it/s]

[{'age': 29, 'region': {'x': 208, 'y': 37, 'w': 120, 'h': 120}, 'gender': {'Woman': 4.4589015146812017e-05, 'Man': 99.99995231628418}, 'dominant_gender': 'Man', 'race': {'asian': 0.32646742183715105, 'indian': 7.5354211032390594, 'black': 0.32972635235637426, 'white': 32.79293179512024, 'middle eastern': 32.33891725540161, 'latino hispanic': 26.676541566848755}, 'dominant_race': 'white', 'emotion': {'angry': 3.388790208301382e-08, 'disgust': 5.378273185664511e-13, 'fear': 9.986144302869349e-06, 'happy': 96.66207989002227, 'sad': 2.6955637382574697e-06, 'surprise': 0.008525826995080203, 'neutral': 3.329385841415625}, 'dominant_emotion': 'happy'}]


## Lookalike search with celebrities (images)

In [22]:
smallest_distance = None

for file in os.listdir("Samples"):
    if file.endswith(".jpg"):
        # result = DeepFace.verify("./Images/person1.jpg", f"Samples/{file}")
        # result = DeepFace.verify("./Images/person2.jpg", f"Samples/{file}")
        result = DeepFace.verify("./Images/match.jpg", f"Samples/{file}")
        if result['verified']:
            print("This person looks exactly like", file.split(".")[0])
            break
        if smallest_distance is None: # First instance
            smallest_distance = (file.split(".")[0], result['distance'])
        else:
            smallest_distance = (file.split(".")[0], result['distance']) if result['distance'] < smallest_distance[1] else smallest_distance
else:
    print(f"No exact match found! Closest match is {smallest_distance[0]}")

No exact match found! Closest match is Hugh Jackman


## Lookalike search with celebrities (video)

In [2]:
# Ruta al directorio que contiene las carpetas de categorías (personas)
carpeta_referencia = "./Samples"

# Obtener la lista de categorías (nombres de personas)
archivos_referencia = os.listdir(carpeta_referencia)

# Inicializar variables para el resultado final
persona_mas_parecida = None
distancia_minima = float('inf')

# Inicializar la captura de la cámara
cap = cv2.VideoCapture(0)

# Capturar el fotograma de la cámara
ret, frame = cap.read()
frame = cv2.resize(frame, (640, 480))

# Guardar la imagen temporal
cv2.imwrite("imagen_temporal.jpg", frame)
imagen_a_comparar = "imagen_temporal.jpg"

# Analizar la imagen para obtener información de edad, género y emoción
info_comparar = DeepFace.analyze(imagen_a_comparar, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
edad_comparar = info_comparar['age']
genero_comparar = info_comparar['dominant_gender']
emocion_comparar = info_comparar['dominant_emotion']

# Iterar sobre cada categoría (persona) en el repositorio
# Iterar sobre cada archivo en la carpeta de referencia
for archivo_referencia in archivos_referencia:
    # Ruta completa a la imagen de referencia de la persona
    imagen_referencia = os.path.join(carpeta_referencia, archivo_referencia)
    imagen_referencia = imagen_referencia.replace("\\", "/")

    # Analizar la imagen de referencia para obtener información de edad, género y emoción
    info_referencia = DeepFace.analyze(imagen_referencia, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
    edad_referencia = info_referencia['age']
    genero_referencia = info_referencia['dominant_gender']

    # Calcular la diferencia en edad y género (otorgando pesos según la importancia)
    diff_edad = abs(edad_comparar - edad_referencia)
    peso_edad = 0.5
    diff_genero = 0 if genero_comparar.lower() == genero_referencia.lower() else 1
    peso_genero = 0.5

    # Realizar la comparación de similitud
    result = DeepFace.verify(imagen_a_comparar, imagen_referencia, model_name='Facenet', distance_metric='euclidean_l2', enforce_detection=False)
    distancia_facial = result["distance"]

    # Calcular la similitud total basada en la diferencia de edad, género y la distancia facial
    similitud = peso_edad * diff_edad + distancia_facial # + peso_genero * diff_genero

    # Actualizar si encontramos una similitud menor
    if similitud < distancia_minima:
        distancia_minima = similitud
        persona_mas_parecida = os.path.splitext(archivo_referencia)[0]  # Eliminar la extensión para obtener la categoría

# Cargar la imagen del actor/actriz más parecido
imagen_act_referencia = cv2.imread(os.path.join(carpeta_referencia, f"{persona_mas_parecida}.jpg"))

# Mostrar la imagen en la parte superior del fotograma
if imagen_act_referencia is not None:
    cv2.imshow('Actor/Actriz Más Parecido(a)', imagen_act_referencia)

# Liberar la captura de la cámara
cap.release()

# Mostrar el resultado en el fotograma
# cv2.putText(frame, f"Persona mas parecida: {persona_mas_parecida}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
# cv2.putText(frame, f"Similitud total: {distancia_minima}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
# cv2.putText(frame, f"Edad: {edad_comparar}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
# cv2.putText(frame, f"Emoción: {emocion_comparar}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
cv2.imshow('Comparación de caras', frame)

# Imprimir el resultado
print(f"La persona más parecida es {persona_mas_parecida} con una similitud total de {distancia_minima}")

# Eliminar la imagen temporal al finalizar
os.remove("imagen_temporal.jpg")

# Esperar hasta que se presione una tecla y cerrar todas las ventanas
cv2.waitKey(0)
cv2.destroyAllWindows()


Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  5.82it/s]


La persona más parecida es Sandra Bullock con una similitud total de 1.316648349942769


## Lookalike search with memes (video)

Filter by distance and then by emotion

In [3]:
# Ruta al directorio que contiene las carpetas de categorías (personas)
reference_folder = "./Datasets/memes/"

# Path to the default image
default_image_path = './Images/default.jpg'

# Image resize dimensions
resize_width = 320
resize_height = 240

# Initialize variables for the final result
most_similar_person = None
min_distance = float('inf')
emocion_diff_anterior = float('inf')
similar_image = None

# Initialize camera capture
cap = cv2.VideoCapture(0)

# Lock for ensuring safe access to shared variables
frame_lock = threading.Lock()
result_lock = threading.Lock()

frame_queue = queue.Queue(maxsize=1)
result_info_queue = queue.Queue(maxsize=1)

# Event variable to notify threads to terminate
exit_event = threading.Event()

# Shared variables between threads
frame = None
result_info = None
index_referencia = 0

def read_camera():
    global frame, most_similar_person, index_referencia
    most_similar_person = None 
    similar_image = None 

    while not exit_event.is_set():
        ret, current_frame = cap.read()
        if ret and current_frame is not None:
            with frame_lock:
                frame = cv2.resize(current_frame, (resize_width, resize_height))
            try:
                frame_queue.put_nowait(frame)
            except queue.Full:
                pass
            cv2.imshow('Camera Frame', frame)

        with result_lock:
            try:
                if most_similar_person is not None:
                    # Si la persona más parecida ha cambiado desde la iteración anterior, cargar la nueva imagen
                    if index_referencia == 0:
                        similar_image_path = os.path.join(reference_folder, f"{most_similar_person}.jpg").replace("\\", "/")
                        similar_image = cv2.resize(cv2.imread(similar_image_path), (resize_width, resize_height))
                    if similar_image is not None:
                        cv2.imshow('Persona más parecida', similar_image)
                else:
                    default_img = cv2.resize(cv2.imread(default_image_path), (resize_width, resize_height))
                    cv2.imshow('Persona más parecida', default_img)
            except Exception as e:
                print(f"Error al mostrar imágenes: {e}")

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def process_image():
    global most_similar_person, min_distance, index_referencia

    while not exit_event.is_set():
        try:
            frame = frame_queue.get_nowait()
        except queue.Empty:
            time.sleep(0.1)
            continue

        cv2.imwrite("temp_image.jpg", frame)
        image_to_compare = "temp_image.jpg"

        # Muestra la imagen que se está comparando en el hilo process_image
        img_compare = cv2.resize(cv2.imread(image_to_compare), (resize_width, resize_height))
        cv2.imshow('Image to Compare in process_image', img_compare)
        cv2.waitKey(1)  # Necesario para que OpenCV muestre la imagen

        if img_compare is None or img_compare.size == 0:
            print("Error: img_compare is None or empty.")
            continue
        # Initialize variables before comparison
        # Analizar emociones primero
        with result_lock:
            min_distance = float('inf')
            emocion_diff_anterior = float('inf')
            most_similar_person = None

        info_comparar = DeepFace.analyze(img_compare, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
        emociones_comparar = info_comparar['emotion']
        emocion_comparar = max(emociones_comparar.values())

        valores_emociones_comparar = list(emociones_comparar.values())
        print(valores_emociones_comparar)
        # Encuentra la posición (índice) de emocion_comparar en la lista de valores
        posicion_emocion_comparar = valores_emociones_comparar.index(emocion_comparar)
        emociones_comparar_lista = list(emociones_comparar.keys())
        emocion_aja = emociones_comparar_lista[posicion_emocion_comparar]
        # Imprime la posición
        print(f"Posición de {emocion_comparar} en la lista de valores: {emocion_aja}")


        # Iterar sobre cada referencia
        for reference_file in os.listdir(reference_folder):
            reference_image_path = os.path.join(reference_folder, reference_file)
            img_reference = cv2.resize(cv2.imread(reference_image_path), (resize_width, resize_height))
            if img_reference is None or img_reference.size == 0:
                print(f"Error: img_reference ({reference_file}) is None or empty.")
                continue

            # Analizar emociones de la imagen de referencia
            info_referencia = DeepFace.analyze(img_reference, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
            emociones_referencia = info_referencia['emotion']
            emocion_referencia = max(emociones_referencia.values())

            # Comparación de emociones antes de la comparación facial
            with result_lock:
                emocion_diff_nueva = abs(emocion_comparar - emocion_referencia)
                if emocion_diff_nueva < emocion_diff_anterior:
                    emocion_diff_anterior = emocion_diff_nueva
                    most_similar_person = os.path.splitext(reference_file)[0]

                    # Comparación facial después de la comparación de emociones
                    result = DeepFace.verify(img_compare, img_reference, model_name='Facenet', distance_metric='euclidean_l2', enforce_detection=False)
                    facial_distance = result["distance"]

                    # Actualizar si se encuentra una distancia facial más pequeña
                    if facial_distance > min_distance:
                        min_distance = facial_distance
                        most_similar_person = os.path.splitext(reference_file)[0]
        
            index_referencia += 1

        # Reiniciar el índice para la próxima iteración
        index_referencia = 0
        # Wait for 5 seconds before the next comparison
        time.sleep(5)

# Start threads
camera_thread = threading.Thread(target=read_camera)
camera_thread.daemon = True
camera_thread.start()

process_thread = threading.Thread(target=process_image)
process_thread.daemon = True
process_thread.start()

# Wait for threads to start completely
time.sleep(2)

# Wait for 'q' key to exit the program
while True:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit_event.set()  # Set the event to notify threads to terminate
        break

# Wait for threads to finish
camera_thread.join()
process_thread.join()

# Release camera capture and close all windows
cap.release()
cv2.destroyAllWindows()

# Eliminar la imagen temporal al finalizar
os.remove("temp_image.jpg")

Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  5.80it/s]


Filter by emotion and then by distance

In [ ]:
# Directory path containing reference images
reference_folder = "./Memes"

# Path to the default image
default_image_path = './default.jpg'

# Image resize dimensions
resize_width = 320
resize_height = 240

# Initialize variables for the final result
most_similar_person = None
min_distance = float('inf')
emocion_diff_anterior = float('inf')
similar_image = None

# Initialize camera capture
cap = cv2.VideoCapture(0)

# Lock for ensuring safe access to shared variables
frame_lock = threading.Lock()
result_lock = threading.Lock()

frame_queue = queue.Queue(maxsize=1)
result_info_queue = queue.Queue(maxsize=1)

# Event variable to notify threads to terminate
exit_event = threading.Event()

# Shared variables between threads
frame = None
result_info = None
index_referencia = 0

def read_camera():
    global frame, most_similar_person, index_referencia
    most_similar_person = None 
    similar_image = None 

    while not exit_event.is_set():
        ret, current_frame = cap.read()
        if ret and current_frame is not None:
            with frame_lock:
                frame = cv2.resize(current_frame, (resize_width, resize_height))
            try:
                frame_queue.put_nowait(frame)
            except queue.Full:
                pass
            cv2.imshow('Camera Frame', frame)

        with result_lock:
            try:
                if most_similar_person is not None:
                    # Si la persona más parecida ha cambiado desde la iteración anterior, cargar la nueva imagen
                    if index_referencia == 0:
                        similar_image_path = os.path.join(reference_folder, f"{most_similar_person}.jpg").replace("\\", "/")
                        similar_image = cv2.resize(cv2.imread(similar_image_path), (resize_width, resize_height))
                    if similar_image is not None:
                        cv2.imshow('Persona más parecida', similar_image)
                else:
                    default_img = cv2.resize(cv2.imread(default_image_path), (resize_width, resize_height))
                    cv2.imshow('Persona más parecida', default_img)
            except Exception as e:
                print(f"Error al mostrar imágenes: {e}")

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def process_image():
    global most_similar_person, min_distance, index_referencia

    while not exit_event.is_set():
        try:
            frame = frame_queue.get_nowait()
        except queue.Empty:
            time.sleep(0.1)
            continue

        cv2.imwrite("temp_image.jpg", frame)
        image_to_compare = "temp_image.jpg"

        # Muestra la imagen que se está comparando en el hilo process_image
        img_compare = cv2.resize(cv2.imread(image_to_compare), (resize_width, resize_height))
        cv2.imshow('Image to Compare in process_image', img_compare)
        cv2.waitKey(1)  # Necesario para que OpenCV muestre la imagen

        if img_compare is None or img_compare.size == 0:
            print("Error: img_compare is None or empty.")
            continue
        # Initialize variables before comparison
        with result_lock:
            min_distance = float('inf')
            emocion_diff_anterior = float('inf')
            most_similar_person = None

        # Iterate over each reference image in the repository
        for reference_file in os.listdir(reference_folder):
            reference_image_path = os.path.join(reference_folder, reference_file)

            # Resize the reference image
            img_reference = cv2.resize(cv2.imread(reference_image_path), (resize_width, resize_height))
            if img_reference is None or img_reference.size == 0:
                print(f"Error: img_reference ({reference_file}) is None or empty.")
                continue

            info_referencia = DeepFace.analyze(img_reference, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
            #edad_referencia = info_referencia['age']
            #genero_referencia = info_referencia['dominant_gender']
            emociones_referencia = info_referencia['emotion']
            emocion_referencia = max(emociones_referencia.values())

            # Perform face recognition similarity comparison
            result = DeepFace.verify(img_compare, img_reference, model_name='Facenet', distance_metric='euclidean_l2', enforce_detection=False)
            
            facial_distance = result["distance"]

            # Update if a smaller distance is found
            with result_lock:
                if facial_distance < min_distance:
                    min_distance = facial_distance
                    info_referencia = DeepFace.analyze(img_reference, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
                    most_similar_person = os.path.splitext(reference_file)[0]
                if facial_distance == min_distance:
                    info_comparar = DeepFace.analyze(img_compare, enforce_detection=False, actions=['age', 'gender', 'emotion'])[0]
                    emociones_comparar = info_comparar['emotion']
                    emocion_comparar = max(emociones_comparar.values())
                    # Obtén la lista de valores (intensidades de emociones)
                    valores_emociones_comparar = list(emociones_comparar.values())
                    print(valores_emociones_comparar)
                    # Encuentra la posición (índice) de emocion_comparar en la lista de valores
                    posicion_emocion_comparar = valores_emociones_comparar.index(emocion_comparar)
                    emociones_comparar_lista = list(emociones_comparar.keys())
                    emocion_aja = emociones_comparar_lista[posicion_emocion_comparar]
                    # Imprime la posición
                    print(f"Posición de {emocion_comparar} en la lista de valores: {emocion_aja}")


                    # Ensure the data types are compatible before subtraction
                    emocion_referencia = float(emocion_referencia)
                    emocion_diff_nueva = abs(emocion_comparar - emocion_referencia)
                    if emocion_diff_nueva > emocion_diff_anterior:
                        print("mejor emocion")
                        emocion_diff_anterior = emocion_diff_nueva
                        most_similar_person = os.path.splitext(reference_file)[0]

            index_referencia += 1

        # Reiniciar el índice para la próxima iteración
        index_referencia = 0
        # Wait for 5 seconds before the next comparison
        time.sleep(5)

# Start threads
camera_thread = threading.Thread(target=read_camera)
camera_thread.daemon = True
camera_thread.start()

process_thread = threading.Thread(target=process_image)
process_thread.daemon = True
process_thread.start()

# Wait for threads to start completely
time.sleep(2)

# Wait for 'q' key to exit the program
while True:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit_event.set()  # Set the event to notify threads to terminate
        break

# Wait for threads to finish
camera_thread.join()
process_thread.join()

# Release camera capture and close all windows
cap.release()
cv2.destroyAllWindows()

os.remove("temp_image.jpg")